In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

In [2]:
cotton_price = 1.5
nitrogen_cost = 0.9
nitrogen_ratio = nitrogen_cost/cotton_price
sq_rate = 100
sq_yield = 1500
yield_sd = 100
yield_noise = 5
eonr_sd = 25
rst_b2g = -0.02
opt_pcts = [50, 75, 100]
rate_rngs = [25, 50, 100]

In [3]:
trial = np.load('../data/Trial_Design.npy')
trial_names = np.load('../data/Trial_Design_names.npy', allow_pickle=True)
trial_idx = [i for i, n in enumerate(trial_names) if not 'rep' in n]
trial_coords = np.load('../data/Trial_Design_coords.npy')
trial_coords_idx = np.int0(np.floor(trial_coords/3))
n = trial_coords.shape[0]

rst_sim = np.load('../data/Trial_sim.npy')
b0_true = rst_sim[:100]
b1_true = rst_sim[100:]

In [5]:
resultsdf = []
for opt_pct in opt_pcts:    
    rst_optr = opt_pct + eonr_sd * b1_true
    rst_optr = np.clip(rst_optr, 0, 200)

    rst_b1 = -2 * rst_b2g * rst_optr + nitrogen_ratio
    rst_b0 = sq_yield + yield_sd * b0_true
    rst_b0 = rst_b0 - (rst_b1 * sq_rate + rst_b2g * sq_rate **2) 
    
    rst_yield_ref =  rst_b0 + rst_b1 * sq_rate + rst_b2g * sq_rate**2
    rst_yield_opt_f =  rst_b0 + rst_b1 * opt_pct + rst_b2g * opt_pct**2
    rst_yield_opt =  rst_b0 + rst_b1 * rst_optr + rst_b2g * rst_optr**2

    rst_net_ref = rst_yield_ref * cotton_price - sq_rate * nitrogen_cost
    rst_net_opt_f = rst_yield_opt_f * cotton_price - opt_pct * nitrogen_cost 
    rst_net_opt = rst_yield_opt * cotton_price - rst_optr * nitrogen_cost 

    rst_net_f = (rst_net_opt_f - rst_net_ref).mean((1,2))
    rst_net_max = (rst_net_opt - rst_net_ref).mean((1,2))


    for idx in tqdm(trial_idx):
        exp_name = trial_names[idx]
        trial_rst = np.nan_to_num(trial[idx])/2

        for rate_rng in rate_rngs:
            
            trial_rate = rate_rng * trial_rst
            inp_rate = trial_rate + sq_rate
            yield_rnd = np.random.randn(*rst_b0.shape) * yield_noise
            rst_yield_obs =  yield_rnd + rst_b0 + rst_b1 * trial_rate
#             rst_yield_obs = yield_rnd + rst_b0 + rst_b1 * trial_rate + rst_b2g * trial_rate**2

            X = trial_rate.reshape(1, -1)
            Xf = (X ** np.array([0,1])[:,None]).T
#             Xf = (X ** np.array([0,1, 2])[:,None]).T
            y = rst_yield_obs.reshape(len(rst_yield_obs), -1).T
            xpred, _, _, _ = np.linalg.lstsq(Xf, y, rcond = None)

            b0_pred = xpred[0,:,None,None]
            b1_pred = xpred[1,:,None,None]
            b2_pred = rst_b2g
#             b2_pred = xpred[2,:,None,None]
            rst_optr_pred = (b1_pred - nitrogen_ratio) / (-2 * b2_pred)
            rst_optr_pred = np.clip(rst_optr_pred, 0, 200)

            rst_yield_obs =  rst_b0 + rst_b1 * inp_rate + rst_b2g * inp_rate**2
            rst_yield_pred =  rst_b0 + rst_b1 * rst_optr_pred + rst_b2g * rst_optr_pred**2

            rst_net_obs = rst_yield_obs * cotton_price - inp_rate * nitrogen_cost 
            rst_net_pred = rst_yield_pred * cotton_price - rst_optr_pred * nitrogen_cost 

            rst_net_loss = (rst_net_obs - rst_net_ref).mean((1,2))
            rst_net_diff = (rst_net_pred - rst_net_ref).mean((1,2))
            rst_net_pct = 100 * rst_net_diff / rst_net_max.mean()
            avg_nrate = rst_optr_pred.mean((1,2))

            rnp = np.stack([rst_net_max, avg_nrate, rst_net_loss, rst_net_diff, rst_net_pct], -1)
            df = pd.DataFrame(rnp, columns=['Max', 'AVG', 'Loss', 'VRT', 'VRTpct'])
            df['rate_rng'] = rate_rng
            df['opt_pct'] = opt_pct
            df['Trial'] = exp_name
            df['Rep'] = np.arange(len(df))
            resultsdf.append(df)

resultsdf = pd.concat(resultsdf).reset_index(drop=True)
resultsdf.to_csv('../data/trial_eco_linf.csv', index = False)